# Probabilistic Time Series Analysis

## Week 5: Hidden Markov Models

---

You will need the following new Python package. Install it either with `conda install -c conda-forge` if you use the Anaconda environment or `pip install`.

    hmmlearn

Places where you are supposed to fill in code are marked

    #
    # TODO: some instructions
    # 
    
The rest of the code we will run and discuss if time permits, otherwise try it out at home and try to answer the questions mentioned in the text boxes for yourself.

### Please turn in the code before 10/10/2018 5:20pm. 

### Your work will be evaluated based on the code and plots. You don't need to write down your answers to other questions in the text blocks, just think them over.

### Title your submission file `lab5-student-[YOUR NET ID].ipynb`.

---

## Setup

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
import random
import pylab
from collections import Counter
from hmmlearn import hmm
import time

## Part I: Data Loading

Load the Wall Street Journal POS dataset. Transform them into indices. 

In [ ]:
train_dir = "../../data/en-wsj-train.pos"
test_dir = "../../data/en-wsj-dev.pos"

def load_pos_data(data_dir, word_indexer=None, label_indexer=None, top_k=20000):
    """
    Function that loads the data
    """
    with open(data_dir, "r") as f:
        # load data
        content = f.readlines()
        intermediate_X = []
        intermediate_z = []
        current_X = []
        current_z = []
        vocab_counter = Counter()
        label_set = set()
        for line in content:
            tokens = line.replace("\n", "").replace("$", "").split("\t")
            if len(tokens) <= 1:
                intermediate_X.append(current_X)
                intermediate_z.append(current_z)
                current_X = []
                current_z = []
            elif len(tokens[1]) > 0:
                vocab_counter[tokens[0]]+=1
                label_set.add(tokens[1])
                current_X.append(tokens[0].lower())
                current_z.append(tokens[1])
        # index data
        top_k_words = vocab_counter.most_common(top_k)
        # 0 is reserved for unknown words
        word_indexer = word_indexer if word_indexer is not None else dict([(top_k_words[i][0], i+1) for i in range(len(top_k_words))])
        word_indexer["UNK"] = 0 
        label_indexer = label_indexer if label_indexer is not None else dict([(label, i) for i, label in enumerate(label_set)])
        output_X = []
        output_z = []
        current_X = []
        current_z = []
        for i in range(len(intermediate_X)):
            for j in range(len(intermediate_X[i])):
                if intermediate_X[i][j] in word_indexer:
                    current_X.append(word_indexer[intermediate_X[i][j]])
                else:
                    current_X.append(0)
                current_z.append(label_indexer[intermediate_z[i][j]])
            # populate holders
            output_X.append(current_X)
            output_z.append(current_z)
            # reset current holder
            current_X = []
            current_z = []
        return output_X, output_z, label_indexer, word_indexer, {v: k for k, v in label_indexer.items()}, {v: k for k, v in word_indexer.items()}

In [ ]:
train_X, train_z, label_indexer, word_indexer, label_lookup, vocab_lookup = load_pos_data(train_dir)
test_X, test_z, _, _, _, _ = load_pos_data(test_dir, word_indexer=word_indexer, label_indexer=label_indexer)

## Part II: HMM Implementation

In this part, you will implement the Hidden Markov Model with following methods:


- sample: given a initial state and the number of steps, returns a sequence of sampled states and observations.


- fit: update the transition matrix, emission matrix, and the initial state probability. Note that in our case, the hidden states are given. We don't need to use EM for the learning. Just count the number of times that given transitions / emissions / initial states appear, and normalize those counts to fill in the parameters.


- decode_single_chain: use the Viterbi Algorithm to find the most probable sequence of states.

In [ ]:
def reconstruct_sequence(idx_list, lookup):
    """
    Function that reconstructs a sequence of index
    """
    return [lookup[x] for x in idx_list]

def percentage_agree(x, y):
    """
    Function that shows the % of agreement among two list
    """
    assert len(x)==len(y)
    return float(np.sum(np.array(x)==np.array(y)))/len(x)

class MyHMM:
    def __init__(self, num_unique_states, num_observations):
        """
        Constructor
        @param num_unique_states: # of unique states (POS Tags)
        @param num_observations: # of unique observations (words)
        """
        self.num_unique_states = num_unique_states
        self.num_observations = num_observations
        self.transition_matrix = np.zeros((num_unique_states, num_unique_states))
        self.emission_matrix = np.zeros((num_unique_states, num_observations))
        self.initial_states_vector = np.zeros(num_unique_states)
    
    def fit(self, X, z):
        """
        Method that fits the model.
        @param X: array-like with dimension [# of examples, # of length]
        @param z: array-like with dimension [# of examples, # of length]
        """
        #
        # TODO: Add your implementation here
        #
        return None
    
    def decode_single_chain(self, x):
        """
        Auxiliary method that uses Viterbi on single chain
        @param X: array-like with dimension [ # of length]
        @return z: array-like with dimension [# of length]
        """
        #
        # TODO: Add your implementation here
        #
        z = []
        return z
        
    def decode(self, X):
        """
        Method that performs the Viterbi the model.
        @param X: array-like with dimension [# of examples, # of length]
        @return z: array-like with dimension [# of examples, # of length]
        """
        return [self.decode_single_chain(sample) for sample in X]
    
    def sample(self, n_step, initial_state):
        """
        Method that given initial state and produces n_step states and observations
        @param n_step: integer
        @param initial_state: an integer indicating the state
        """
        #
        # TODO: Add your implementation here
        #
        states = []  # aka z
        observations = []  # aka x
        return states, observations

## Part III: Validation

### Learn an HMM

In [ ]:
num_states = len(label_indexer)
num_obs = len(word_indexer)
my_hmm = MyHMM(num_states, num_obs)
my_hmm.fit(train_X, train_z)

### Decode

In [ ]:
i = 5
res = my_hmm.decode_single_chain(test_X[i])
print("data: {0} \n pred: {1} \n label: {2}".format(reconstruct_sequence(test_X[i], vocab_lookup),
                                                    reconstruct_sequence(res, label_lookup),
                                                  reconstruct_sequence(test_z[i], label_lookup)))

In [ ]:
start_time = time.time()
pred_train = my_hmm.decode(train_X[:1000])
print("takes {0} seconds".format(time.time() - start_time))

In [ ]:
start_time = time.time()
pred_test = my_hmm.decode(test_X)
print("takes {0} seconds".format(time.time() - start_time))

In [ ]:
np.mean([percentage_agree(pred_train[i], train_z[i]) for i in range(len(pred_train))])

In [ ]:
np.mean([percentage_agree(pred_test[i], test_z[i]) for i in range(len(pred_test))])

### Sample

In [ ]:
pos_tag, words = my_hmm.sample(10, label_indexer["NNP"])
print(reconstruct_sequence(pos_tag, label_lookup))
print(reconstruct_sequence(words, vocab_lookup))